In [4]:
#Using word lists from www.sketchengine.eu

from __future__ import unicode_literals,print_function,division
from io import open
import glob
import os

def findFiles(path): return glob.glob(path)

print(findFiles('data/words/*.csv'))

import unicodedata
import string

all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters) # the number or letters in the given ascii table (57 total letters)

# Turn a Unicode string to plain ASCII, thanks to https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

print(unicodeToAscii('Ślusàrski'))

# Build the category_lines dictionary, a list of names per language
category_lines = {}
all_categories = []

# Read a file and split into lines
def readLines(filename):
    lines = open(filename, encoding='ANSI').read().strip().split('\n')
    return [unicodeToAscii(line) for line in lines]

for filename in findFiles('data/words/*.csv'):
    category = os.path.splitext(os.path.basename(filename))[0]
    all_categories.append(category)
    lines = readLines(filename)
    category_lines[category] = lines

n_categories = len(all_categories)


['data/words\\english.csv', 'data/words\\french.csv', 'data/words\\german.csv', 'data/words\\italian.csv', 'data/words\\spanish.csv']
Slusarski


In [5]:
print(category_lines['english'][:5])

['the', 'and', 'to', 'of', 'a']


In [9]:
import torch 

# transforms a letter to a number "a" = 0
def letterToIndex(letter):
    return all_letters.find(letter)

def letterToTensor(letter):
    tensor = torch.zeros(1, n_letters)
    tensor[0][letterToIndex(letter)] = 1
    return tensor

#Turn a string of letters into a tensor of size <word_length x 1 x n_letters>
def lineToTensor(line):
    tensor = torch.zeros(len(line),1,n_letters)
    for li, letter in enumerate(line):
        tensor[li][0][letterToIndex(letter)] = 1
    return tensor

print(letterToTensor('ò'))
print(lineToTensor('Scòtt').size())

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 1.]])
torch.Size([5, 1, 57])
